### Assignment-01-附加题-极具挑战性的题目，基础一般的同学请把前边的做好就非常非常好了，这个是一个趣味题 😝

## 基于模式匹配的对话机器人实现

### Pattern Match

机器能否实现对话，这个长久以来是衡量机器人是否具有智能的一个重要标志。 Alan Turing早在其文中就提出过一个测试机器智能程度的方法，该方法主要是考察人类是否能够通过对话内容区分对方是机器人还是真正的人类，如果人类无法区分，我们就称之为具有”智能“。而这个测试，后来被大家叫做”图灵测试“，之后也被翻拍成了一步著名电影，叫做《模拟游戏》。 



既然图灵当年以此作为机器是否具备智能的标志，这项任务肯定是复杂的。自从 1960s 开始，诸多科学家就希望从各个方面来解决这个问题，直到如今，都只能解决一部分问题。 目前对话机器人的建立方法有很多，今天的作业中，我们为大家提供一共快速的基于模板的对话机器人配置方式。

此次作业首先希望大家能够读懂这段程序的代码，其次，在此基于我们提供的代码，**能够把它改造成汉语版本，实现对话效果。**

```
Pattern: (我想要A)
Response: (如果你有 A，对你意味着什么呢？)

Input: (我想要度假)
Response: (如果你有度假，对你意味着什么呢？)
```

为了实现模板的判断和定义，我们需要定义一个特殊的符号类型，这个符号类型就叫做"variable"， 这个"variable"用来表示是一个占位符。例如，定义一个目标: "I want X"， 我们可以表示成  "I want ?X", 意思就是?X是一个用来占位的符号。

如果输入了"I want holiday"， 在这里 'holiday' 就是 '?X'

In [1]:
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

In [2]:
def pat_match(pattern, saying):
    if is_variable(pattern[0]): return True
    else:
        if pattern[0] != saying[0]: return False
        else:
            return pat_match(pattern[1:], saying[1:])

### 例如

In [3]:
pat_match('I want ?X'.split(), "I want holiday".split())

True

In [4]:
pat_match('I have dreamed a ?X'.split(), "I dreamed about dog".split())

False

In [5]:
pat_match('I dreamed about ?X'.split(), "I dreamed about dog".split())

True

In [6]:
print(pat_match('?X I dreamed a dog'.split(),"I want holiday".split())) # ???

# Should be False, right?

# 按照给定代码示例，如果 pattern 以 ?X 起始，那么无论后面是什么，pat_match 都会返回 True
# 但这两个句子表达完全不同的意思，不应认为匹配。

# 基于以上问题做了一点改动：但还是有非常大的限制：
# 1. 规定 pattern 与 saying 等长（同等数量words/token）
# 2. 判断剔除掉 pattern中 ?X 及 saying中对应位置词后剩余字符串是否相同-->是否match

def pat_match(pattern, saying):
    
    if len(pattern) != len(saying):
        return False
    
    else:
        idx = [is_variable(s) for s in pattern]

        pattern = [p for i,p in enumerate(pattern) if idx[i] == False]
        saying =  [s for i,s in enumerate(saying) if idx[i] == False]   

        return pattern == saying

    
# print(pat_match('I dreamed about ?X'.split(), "I dreamed about dog".split()))
# print(pat_match('I have dreamed a ?X'.split(), "I dreamed about dog".split()))


print("Examples I think they shouldn't be matched:")
print(pat_match("?X".split(),"I dreamed a dog".split())) 
print(pat_match('?X I dreamed a dog'.split(),"?X I want holiday".split()))

print("Examples I think they should be matched:")
print(pat_match("?X dreamed a dog".split(),"I dreamed a dog".split())) 
print(pat_match("I ?X a dog".split(),"I dreamed a dog".split()))
print(pat_match("?X".split(),"dog".split()))


# btw. 对于习惯使用 Matlab & R indexing 的人来说，python 的操作真是令人窒息。
# 难道不能直接用 logical array 作为 index ??

True
Examples I think they shouldn't be matched:
False
False
Examples I think they should be matched:
True
True
True


In [7]:
# 然而以上没有解决不等长的问题，现在把匹配目标改成：
# pattern: I dreamed ?X yesterday; 
# saying: I dreamed blahblahblah (any length of words here) yesterday. are matched

def pat_match(pattern, saying):
    
    idx = [is_variable(p) for p in pattern]
    var = [v for i,v in enumerate(pattern) if idx[i] == True] # ?X
    pat = [p for i,p in enumerate(pattern) if idx[i] == False] # strings in pattern
    matched = (var[0], [s for s in saying if s not in pat]) # strings corresponding to ?X
    
    return matched


print(pat_match("I dreamed ?X yesterday".split(),"I dreamed a dream yesterday".split()))
print(pat_match("?X dreamed holiday".split(),"my friends dreamed holiday".split()))
print(pat_match("I ate ?X".split(),"I ate apples,bananas,oranges".split()))

('?X', ['a', 'dream'])
('?X', ['my', 'friends'])
('?X', ['apples,bananas,oranges'])


### 获得匹配的变量

以上的函数能够判断两个 pattern 是不是相符，但是我们更加希望的是获得每个variable对应的是什么值。

我们对程序做如下修改:

In [8]:
def pat_match(pattern, saying):
    if is_variable(pattern[0]):
        return pattern[0], saying[0]
    else:
        if pattern[0] != saying[0]: return False
        else:
            return pat_match(pattern[1:], saying[1:])

In [9]:
pat_match("?X equals ?X".split(), "2+2 equals 2+2".split())

('?X', '2+2')

但是，如果我们的 Pattern 中具备两个变量，那么以上程序就不能解决了，我们可以对程序做如下修改: 


In [10]:
def pat_match(pattern, saying):
    if not pattern or not saying: return []
    
    if is_variable(pattern[0]):
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]: return []
        else:
            return pat_match(pattern[1:], saying[1:])

于是，我们可以获得： 

In [11]:
pat_match("?X greater than ?Y".split(), "3 greater than 2".split())

[('?X', '3'), ('?Y', '2')]

In [12]:
pat_match("?X greater than ?Y".split(), "I like to eat banana".split()) 
# 同样问题当 ?X 出现在句首。。。这里不改了，改最终版的

[('?X', 'I')]

为了方便接下来的替换工作，我们新建立两个函数，一个是把我们解析出来的结果变成一个 dictionary，一个是依据这个 dictionary 依照我们的定义的方式进行替换。

In [15]:
def pat_to_dict(patterns):
    return {k: v for k, v in patterns}

In [16]:
def subsitite(rule, parsed_rules):
    if not rule: return []
    
    return [parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules)

In [19]:
got_patterns = pat_match("I want ?X".split(), "I want iPhone".split())
print(got_patterns)

[('?X', 'iPhone')]


In [20]:
subsitite("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns))

['What', 'if', 'you', 'mean', 'if', 'you', 'got', 'a', 'iPhone']

为了将以上输出变成一句话，也很简单，我们使用 Python 的 join 方法即可： 

In [21]:
john_pat = pat_match('?P needs ?X'.split(), "John needs resting".split())

In [22]:
' '.join(subsitite("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns)))

'What if you mean if you got a iPhone'

In [23]:
john_pat = pat_match('?P needs ?X'.split(), "John needs vacation".split())

In [24]:
subsitite("Why does ?P need ?X ?".split(), pat_to_dict(john_pat))

['Why', 'does', 'John', 'need', 'vacation', '?']

In [25]:
' '.join(subsitite("Why does ?P need ?X ?".split(), pat_to_dict(john_pat)))

'Why does John need vacation ?'

那么如果我们现在定义一些patterns，就可以实现基于模板的对话生成了:

In [26]:
defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}

In [27]:
import random

def get_response(saying, rules):
    """" please implement the code, to get the response as followings:
    
    >>> get_response('I need iPhone') 
    >>> Image you will get iPhone soon
    >>> get_response("My mother told me something")
    >>> Talk about more about your monther.
    """
    
    for probe in rules.keys():
        got_patterns = pat_match(probe.split(),saying.split())
        
        if got_patterns:
            resp_ = random.choice(rules[probe])
            print(' '.join(subsitite(resp_.split(), pat_to_dict(got_patterns))))
    

get_response('I need iPhone',defined_patterns)

get_response("My mother told me something",defined_patterns)

Image you will get iPhone soon
Talk about more about your mother


> **评阅点**: 运行代码，是否能够实现基本的语言问答

### Segment Match

我们上边的这种形式，能够进行一些初级的对话了，但是我们的模式逐字逐句匹配的， "I need iPhone" 和 "I need ?X" 可以匹配，但是"I need an iPhone" 和 "I need ?X" 就不匹配了，那怎么办？ 

为了解决这个问题，我们可以新建一个变量类型 "?\*X", 这种类型多了一个星号(\*),表示匹配多个

首先，和前文类似，我们需要定义一个判断是不是匹配多个的variable

In [28]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

In [29]:
is_pattern_segment('?*P')

True

In [30]:
from collections import defaultdict

然后我们把之前的 ```pat_match```程序改写成如下， 主要是增加了 ``` is_pattern_segment ```的部分. 

In [31]:
# fail = [True, None]
fail = [None]

def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    
    pat = pattern[0]
    
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail

这段程序里比较重要的一个新函数是 ```segment_match```，这个函数输入是一个以 ```segment_pattern```开头的模式，尽最大可能进行，匹配到这个*边长*的变量对于的部分。

In [32]:
def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')

    if not rest: return (seg_pat, saying), len(saying)    
    
    for i, token in enumerate(saying):
        if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
            return (seg_pat, saying[:i]), i
    
    return None,len(saying) # (seg_pat, saying), len(saying)

# if segment_match return (seg_pat, saying), len(saying) in the end,
# when ?*X appears at the beginning of the pattern, it got matched no matter what


def is_match(rest, saying):
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])


print(pat_match_with_seg("?*X was surprised".split(),"I was surprised".split()))
print(pat_match_with_seg("?*X was happy".split(),"I was surprised".split()))


[('?X', ['I'])]
[None]


In [33]:
segment_match('?*P is very good'.split(), "My dog and my cat is very good".split())

(('?P', ['My', 'dog', 'and', 'my', 'cat']), 5)

现在，我们就可以做到以下的匹配模式了: 

In [34]:
pat_match_with_seg('?*P is very good and ?*X'.split(), "My dog is very good and my cat is very cute".split())

[('?P', ['My', 'dog']), ('?X', ['my', 'cat', 'is', 'very', 'cute'])]

如果我们继续定义一些模板，我们进行匹配，就能够进行更加复杂的问题了: 

In [35]:
response_pair = {
    'I need ?X': [
        "Why do you neeed ?X"
    ],
    "I dont like my ?X": ["What bad things did ?X do for you?"]
}

In [36]:
pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())

[('?X', ['an', 'iPhone'])]

In [37]:
subsitite("Why do you need ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))

['Why', 'do', 'you', 'need', ['an', 'iPhone']]

 我们会发现，pat_to_dict在这个场景下会有有一点小问题，没关系，修正一些: 

In [38]:
def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

In [39]:
subsitite("Why do you need ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))

['Why', 'do', 'you', 'need', 'an iPhone']

如果我们定义这样的一个模板:

In [40]:
("?*X hello ?*Y", "Hi, how do you do")

('?*X hello ?*Y', 'Hi, how do you do')

In [41]:
subsitite("Hi, how do you do?".split(), pat_to_dict(pat_match_with_seg('?*X hello ?*Y'.split(), 
                  "I am mike, hello ".split())))

['Hi,', 'how', 'do', 'you', 'do?']

### 现在是你的时间了

In [43]:
#我们给大家一些例子: 
    
rules = {
    "?*X hello ?*Y": ["Hi, how do you do?"],
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."]
}

import random

def get_response(saying, rules):

    for probe in rules.keys():
        got_patterns = pat_match_with_seg(probe.split(),saying.split())
        
        if got_patterns != fail:
            resp_ = random.choice(rules[probe])
            print(' '.join(subsitite(resp_.split(), pat_to_dict(got_patterns))))

            
get_response("hello Im Mike",rules)   

get_response("I was surprised",rules) 



Hi, how do you do?
Were you really surprised ?


### 问题1

编写一个程序, ```get_response(saying, response_rules)```输入是一个字符串 + 我们定义的 rules，例如上边我们所写的 pattern， 输出是一个回答。

> **评阅点**： get_response是否定义完整，功能是否齐备

### 问题2

改写以上程序，将程序变成能够支持中文输入的模式。
*提示*: 你可以需用用到 jieba 分词

> **评阅点**： 是否能够支持中文问题，jieba分词的使用

In [59]:
# !pip install jieba

  Created wheel for jieba: filename=jieba-0.42-cp37-none-any.whl size=19314465 sha256=fc5d964036628a7e332fadf4d8c2690553ba9bd81be0e58a317a54e203e1539e
  Stored in directory: C:\Users\lab\AppData\Local\pip\Cache\wheels\9b\78\fe\c177aa424d4692d4f55ff59b053e3f52c784401a78f0616232
Successfully built jieba


In [45]:
rules = {
    "?*X 你好 ?*Y": ["你好，好久不见。"],
    "我感到 ?*X": ["哦，你真的感到 ?X 吗?", "哈哈，我就知道你一定会感到 ?X 。"],
    '?*x 我想 ?*y': ['你觉得 ?y 有什么意义呢？', '为什么你想 ?y', '你可以想想你很快就可以 ?y 了'],
    
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon']
}

import re
import jieba
import random

def token(string):
    return re.findall('\w+',string)

def cut(string):
    return list(jieba.cut(string))

def flatten_list(nested_list):
    return [item for items in nested_list for item in items]


def get_response_CH(saying, rules):

    saying = cut(saying)
    saying = [s for s in saying if s.isalpha()]
    
    for probe in rules.keys():
        pat = flatten_list([cut(''.join(token(p))) if not is_pattern_segment(p) else [p] for p in probe.split()])
        got_patterns = pat_match_with_seg(pat,saying)
        
        if got_patterns != fail:
            resp = random.choice(rules[probe]).split()
            resp = ' '.join(subsitite(resp, pat_to_dict(got_patterns)))
            if not all([ord(s)>=65 and ord(s)<=122 for s in "".join(saying)]):
                resp = "".join(resp.split())
            print(resp)
            
#             if all([ord(s)>=65 and ord(s)<=122 for s in "".join(saying)]):    # English
#                 print(' '.join(subsitite(resp, pat_to_dict(got_patterns))))
#             else:
#                 print(''.join(subsitite(resp, pat_to_dict(got_patterns))))     # Chinese


get_response_CH("我感到快乐",rules)   

get_response_CH("好久不见，你好吗？",rules)

get_response_CH("I want a new Mac", rules)

get_response_CH("hello how are you?",rules)

get_response_CH("我想学习画画",rules)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lab\AppData\Local\Temp\jieba.cache
Loading model cost 0.892 seconds.
Prefix dict has been built successfully.


哦，你真的感到快乐吗?
你好，好久不见。
Why do you want a new Mac
Please state your problem
你可以想想你很快就可以学习画画了


### 问题3

多设计一些模式，让这个程序变得更好玩，多和大家交流，看看大家有什么好玩的模式

编程能力实在有限，这个暂时做不到。希望结课的时候可以。

> **评阅点**: 是否设计了更有趣的点？

### 问题4

1. 这样的程序有什么优点？有什么缺点？你有什么可以改进的方法吗？ 
2. 什么是数据驱动？数据驱动在这个程序里如何体现？
3. 数据驱动与 AI 的关系是什么？ 


1. 优点是简洁吧，基于模板匹配这样一个规则可以生成任何对话。而且由于dictionary里面的对应规则是人事先写好的，回复可以很人性化/个性化。
缺点也很明显，需要事先设计好rules, 要考虑到对方可能说哪些话，这个就太有局限性了。一个是对方说的话可能完全不在定义范围内，另一个是模板匹配基于具体的单词而不是语义，那么对方说的话一旦有一些变体，匹配就会出问题。对于第一个问题（对方说了不在范围内的话），当然最简单粗暴的做法就是回复所有不在定义范围内的语句：我不理解您在说什么。但是我想无论是这一种还是对方说了摸棱两可或者颠三倒四的话，能使机器在所有情况下（而非大多数预设的情况）像人一样回复的话还是需要使机器先进行一步类似于人一样的语义理解。不是逐词处理，而是把对方说的话先抽提出有效信息，映射到特定的语义空间（有层级，可以通过学习扩增的一个域），再生成回复。举个例子，如果我说：我很开心，那么机器首先要做的是抽提出开心这个关键词，把它投射到一个语义空间比如我们把它设定为一个关于心情或者情绪的集合（相当于机器“理解”了我现在说的这句话跟情绪表达有关），然后再往下一层，把这个词投射到积极情绪的子集,etc. 那么它可以在相关的域或者集合内搜索回复了。如果我说：我今天跑步去了。机器知道我表达的意思和运动有关，那么它可以搜索和运动相关的信息作为回复。对方表达得少，给的信息少，机器能选择回答的自由度大但是不确定性也大；对方表达得越多，机器能搜集到的信息也就越多，比如我既说了我很开心也说了我今天跑步去了，那么机器的回答可以基于二者的一个联系生成。这和人的思维是类似的，基于关联（association).虽然和当前程序一样，需要事先生成一个映射空间，但是句子的表达是无穷尽的，利用穷举法是没有办法覆盖所有情况的；然而语义范畴却是非常有限的，特别是当给定层级的时候，最上层的只可能有很少几类，而它们也正是最重要的。越往下越细微末节，是不需要耗费太多资源处理的。

2. 在我的理解里面数据驱动和假设驱动或者理论驱动是两个反向或者互补的过程：假设/理论驱动就是先提出一个理论框架，然后再拿数据去验证。数据驱动是不基于先验假设，直接从数据种挖掘规律，得出结论或者新的理论。在这个程序中我想应该是体现在回复是没有基于太多的假设和语义规则设定的，机器会做出什么回复取决于用户（输入数据）说了什么。

3. 数据驱动推动了AI的发展，最早的AI是由各个领域的专家写的专家系统，完全按照既定规则在运作。这种写死的instruction table的方式非常有局限性，因为只能处理预先写好的特定问题，不能generalize, 不能处理新的问题。数据驱动能推动AI再度兴起的原因是这种方式能让机器处理问题的方式更灵活，机器可以通过学习进行泛化，知识迁移，等等。数据包含了所有信息，而人能从中提炼出来的理论知识太少，与其把人知道的告诉机器，不如授之以渔让其自学。而且不仅是AI, 人要进步也得数据驱动。

> **评阅点**: 是否提出了比较有意义的问题？对于数据驱动是否理解到位？

一些参考 pattern

In [182]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}



很有趣
